# Metodología DiD de CodeInterpreter

# Librerías

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pycountry
from countryinfo import CountryInfo as CInfo # Se conecta con la informacion de cada país
import matplotlib.pyplot as plt
import  altair  as  alt # Graficos de promedios
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import  altair  as  alt

# Read Data

In [3]:
data = pd.read_csv("https://raw.githubusercontent.com/github/innovationgraph/main/data/languages.csv", delimiter=',')
data

,num_pushers,language,language_type,iso2_code,year,quarter
0,2066,HTML,markup,AE,2020,1
1,1627,CSS,markup,AE,2020,1
2,288,Jupyter Notebook,markup,AE,2020,1
3,108,Vue,markup,AE,2020,1
4,1734,JavaScript,programming,AE,2020,1
...,...,...,...,...,...,...
104437,198,TypeScript,programming,ZW,2023,4
104438,150,C,programming,ZW,2023,4
104439,140,Dockerfile,programming,ZW,2023,4
104440,140,PHP,programming,ZW,2023,4


In [4]:
# Filtro de EU
data = data[data.iso2_code != "EU"]

# Filtro de XK = Kosovo
data = data[data.iso2_code != "XK"]

In [5]:
# This will give you the count of rows with NaN values.
nan_rows_count = data.isna().any(axis=1).sum()
print(f"There are {nan_rows_count} rows with NaN values in the dataset.")

There are 35 rows with NaN values in the dataset.


In [6]:
data[data["iso2_code"].isnull()] = "NA"

C:\Users\ronco\AppData\Local\Temp\ipykernel_19924\143898119.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NA' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data[data["iso2_code"].isnull()] = "NA"
C:\Users\ronco\AppData\Local\Temp\ipykernel_19924\143898119.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NA' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data[data["iso2_code"].isnull()] = "NA"
C:\Users\ronco\AppData\Local\Temp\ipykernel_19924\143898119.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NA' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data[data["iso2_code"].isnull()] = "NA"


## Keep the 10 most used programming languages

In [7]:
top_program_lang = programming_languages = [
    "Python", "Java",
    "C++", "JavaScript", "C#",
    "Ruby", "Swift", 
    "Kotlin", "PHP", "TypeScript"
]

In [8]:
data2 = data[data['language'].isin(top_program_lang)]
data2 = data2.reset_index(drop=True)
data2

,num_pushers,language,language_type,iso2_code,year,quarter
0,1734,JavaScript,programming,AE,2020,1
1,816,Python,programming,AE,2020,1
2,685,Java,programming,AE,2020,1
3,377,Ruby,programming,AE,2020,1
4,354,PHP,programming,AE,2020,1
...,...,...,...,...,...,...
17670,774,JavaScript,programming,ZW,2023,4
17671,391,Python,programming,ZW,2023,4
17672,198,TypeScript,programming,ZW,2023,4
17673,140,PHP,programming,ZW,2023,4


In [9]:
data2['year_quarter'] = data2['year'].astype(str) + '-Q' + data2['quarter'].astype(str)
data2
# Reseteamos el index
data2 = data2.reset_index(drop=True)
data2

,num_pushers,language,language_type,iso2_code,year,quarter,year_quarter
0,1734,JavaScript,programming,AE,2020,1,2020-Q1
1,816,Python,programming,AE,2020,1,2020-Q1
2,685,Java,programming,AE,2020,1,2020-Q1
3,377,Ruby,programming,AE,2020,1,2020-Q1
4,354,PHP,programming,AE,2020,1,2020-Q1
...,...,...,...,...,...,...,...
17670,774,JavaScript,programming,ZW,2023,4,2023-Q4
17671,391,Python,programming,ZW,2023,4,2023-Q4
17672,198,TypeScript,programming,ZW,2023,4,2023-Q4
17673,140,PHP,programming,ZW,2023,4,2023-Q4


In [10]:
# Creating a unique identifier
data2['unique_id'] = data2['iso2_code'] + '-' + data2['language']


In [11]:
import pandas as pd
import numpy as np

# Assume 'df' is your original DataFrame after reading the CSV file.
# 'unique_id' and 'year_quarter' are columns in your DataFrame.

# Create a DataFrame of unique identifiers
iso2_code = pd.DataFrame({'iso2_code': data2['iso2_code'].unique()})

# Create a DataFrame of unique identifiers for languages 
language = pd.DataFrame({'language': data2['language'].unique()})

# Create a DataFrame of all time periods
# time_periods = pd.DataFrame({'year_quarter': range(data2['year_quarter'].min(), data2['year_quarter'].max() + 1)})
year_quarter = pd.DataFrame({'year_quarter': data2['year_quarter'].unique()})

# Create the Cartesian product of unique_ids and time_periods
balanced_panel = iso2_code.merge(language, how='cross').merge(year_quarter, how='cross')

balanced_panel["unique_id"] = balanced_panel["iso2_code"] + "-" +balanced_panel["language"]
balanced_panel

# # Merge the balanced panel with the original data
balanced_df = balanced_panel.merge(data2, on=['unique_id', 'year_quarter'], how='left')

# # Merge the DataFrames with suffixes
balanced_df = balanced_panel.merge(data2, on=['unique_id', 'year_quarter'], how='left', suffixes=('', '_y'))

# # Now, drop the columns with '_y' suffix, which are from the right DataFrame
balanced_df = balanced_df.loc[:, ~balanced_df.columns.str.endswith('_y')]
balanced_df 


,iso2_code,language,year_quarter,unique_id,num_pushers,language_type,year,quarter
0,AE,JavaScript,2020-Q1,AE-JavaScript,1734,programming,2020,1
1,AE,JavaScript,2020-Q2,AE-JavaScript,2299,programming,2020,2
2,AE,JavaScript,2020-Q3,AE-JavaScript,2366,programming,2020,3
3,AE,JavaScript,2020-Q4,AE-JavaScript,2468,programming,2020,4
4,AE,JavaScript,2021-Q1,AE-JavaScript,2544,programming,2021,1
...,...,...,...,...,...,...,...,...
25595,LS,Kotlin,2022-Q4,LS-Kotlin,NaN,NaN,NaN,NaN
25596,LS,Kotlin,2023-Q1,LS-Kotlin,NaN,NaN,NaN,NaN
25597,LS,Kotlin,2023-Q2,LS-Kotlin,NaN,NaN,NaN,NaN
25598,LS,Kotlin,2023-Q3,LS-Kotlin,NaN,NaN,NaN,NaN


In [12]:
# Function to convert quarter format to integer
def quarter_to_int(quarter_string):
    year, q = quarter_string.split('-')
    year = int(year)
    quarter_number = int(q[1])  # Q1, Q2, Q3, Q4 -> 1, 2, 3, 4
    base_year = 2020  # Adjust based on your balanced_df, or set dynamically
    return 4 * (year - base_year) + quarter_number

# Applying the function
balanced_df['quarter'] = balanced_df['year_quarter'].apply(quarter_to_int)
balanced_df['year'] = balanced_df['year_quarter'].str.split('-').str[0]
balanced_df.loc[balanced_df["num_pushers"].isnull(), "num_pushers"] = 0
balanced_df

,iso2_code,language,year_quarter,unique_id,num_pushers,language_type,year,quarter
0,AE,JavaScript,2020-Q1,AE-JavaScript,1734,programming,2020,1
1,AE,JavaScript,2020-Q2,AE-JavaScript,2299,programming,2020,2
2,AE,JavaScript,2020-Q3,AE-JavaScript,2366,programming,2020,3
3,AE,JavaScript,2020-Q4,AE-JavaScript,2468,programming,2020,4
4,AE,JavaScript,2021-Q1,AE-JavaScript,2544,programming,2021,5
...,...,...,...,...,...,...,...,...
25595,LS,Kotlin,2022-Q4,LS-Kotlin,0,NaN,2022,12
25596,LS,Kotlin,2023-Q1,LS-Kotlin,0,NaN,2023,13
25597,LS,Kotlin,2023-Q2,LS-Kotlin,0,NaN,2023,14
25598,LS,Kotlin,2023-Q3,LS-Kotlin,0,NaN,2023,15


## Per capita measures

In [13]:
# Definimos una función que convierte los nombres de países en códigos iso2
def country_to_iso2(country_name):
    try:
        # Intentamos obtener el código iso2 del país usando pycountry
        return pycountry.countries.get(name=country_name).alpha_2
    except AttributeError:
        try:
            # Manejamos casos especiales donde el nombre del país no coincide exactamente con la base de datos de pycountry
            special_cases = {
                "Czechia (Czech Republic)": "CZ",
                "Congo (Congo-Brazzaville)": "CG",
                "Holy See": "VA",
                "Timor-Leste (East Timor)": "TL",
                "Ukraine (with certain exceptions)": "UA",
                "Taiwan": "TW",
                "Bolivia": "BO",
                "Tanzania": "TZ",
                "South Korea": "KR",
                "Moldova": "MD",
                "Brunei": "BN"
            }
            return special_cases[country_name]
        except KeyError:
            return None

# Creamos una lista de países y obtenemos sus códigos iso2 usando la función country_to_iso2
gpt_countries_list = [
    "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", "Argentina", "Armenia", "Australia", "Austria",
    "Azerbaijan", "Bahamas", "Bangladesh", "Barbados", "Belgium", "Belize", "Benin", "Bhutan", "Bolivia",
    "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria", "Burkina Faso", "Cabo Verde", "Canada",
    "Chile", "Colombia", "Comoros", "Congo (Congo-Brazzaville)", "Costa Rica", "Côte d'Ivoire", "Croatia", "Cyprus",
    "Czechia", "Denmark", "Djibouti", "Dominica", "Dominican Republic", "Ecuador", "El Salvador", "Estonia", "Fiji",
    "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany", "Ghana", "Greece", "Grenada", "Guatemala", "Guinea",
    "Guinea-Bissau", "Guyana", "Haiti", "Holy See", "Honduras", "Hungary", "Iceland", "India", "Indonesia", "Iraq",
    "Ireland", "Israel", "Italy", "Jamaica", "Japan", "Jordan", "Kazakhstan", "Kenya", "Kiribati", "Kuwait",
    "Kyrgyzstan", "Latvia", "Lebanon", "Lesotho", "Liberia", "Liechtenstein", "Lithuania", "Luxembourg", "Madagascar",
    "Malawi", "Malaysia", "Maldives", "Mali", "Malta", "Marshall Islands", "Mauritania", "Mauritius", "Mexico",
    "Micronesia", "Moldova", "Monaco", "Mongolia", "Montenegro", "Morocco", "Mozambique", "Myanmar", "Namibia",
    "Nauru", "Nepal", "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria", "North Macedonia", "Norway",
    "Oman", "Pakistan", "Palau", "Palestine, State of", "Panama", "Papua New Guinea", "Paraguay", "Peru", "Philippines",
    "Poland", "Portugal", "Qatar", "Romania", "Rwanda", "Saint Kitts and Nevis", "Saint Lucia",
    "Saint Vincent and the Grenadines", "Samoa", "San Marino", "Sao Tome and Principe", "Saudi Arabia", "Senegal",
    "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Slovakia", "Slovenia", "Solomon Islands", "South Africa",
    "South Korea", "Spain", "Sri Lanka", "Suriname", "Sweden", "Switzerland", "Taiwan", "Tanzania", "Thailand",
    "Timor-Leste", "Togo", "Tonga", "Trinidad and Tobago", "Tunisia", "Turkey", "Tuvalu", "Uganda", "Ukraine",
    "United Arab Emirates", "United Kingdom", "United States", "Uruguay", "Vanuatu", "Zambia"
]

gpt_countries_iso = [country_to_iso2(country) for country in gpt_countries_list]

# Agregamos una nueva columna gpt_available, con valor 1 si el código iso2 está en la lista gpt_countries_iso, y 0 en caso contrario
balanced_df["gpt_available"] = balanced_df["iso2_code"].apply(lambda row: 1 if row in gpt_countries_iso else 0)



In [14]:
countries = data2.iso2_code.unique()

def create_populations_dictionary():
    country_populations = {}
    special_cases = {"MM": 54688774, "PS": 5483450, "ME": 602445, "AD":79824}
    # special_cases = {}
    for country in countries:
        try:
            country_populations.update({country: CInfo(country).info()["population"]})
        except KeyError:
            try:
                fallback_name = pycountry.countries.lookup(country).name
                country_populations.update({country: CInfo(fallback_name).info()["population"]})
            except KeyError:
                print(country)
                country_populations.update({country: special_cases[country]})

    return country_populations

country_populations = create_populations_dictionary()

ME
MM
PS
AD


# Creación de variables "population", "pushes_pc", "time", "Post" & "Treatment"

**population**

In [15]:
balanced_df["population"] = balanced_df["iso2_code"].map(country_populations)

**pushes_pc**

In [16]:
balanced_df["num_pushers_pc"] = (balanced_df["num_pushers"] / balanced_df["population"])*100000

**time**

In [17]:
balanced_df.num_pushers_pc.describe()

count     25600.0
unique    16759.0
top           0.0
freq       7925.0
Name: num_pushers_pc, dtype: float64

**Post**



In [18]:
balanced_df.year = balanced_df.year.astype(int)

In [19]:
# Aplicar la condición y crear la columna post_CI
balanced_df['post_gpt'] = balanced_df['quarter'] >= 12
balanced_df['post_gpt'] = balanced_df['post_gpt'].astype(int)
balanced_df

,iso2_code,language,year_quarter,unique_id,num_pushers,language_type,year,quarter,gpt_available,population,num_pushers_pc,post_gpt
0,AE,JavaScript,2020-Q1,AE-JavaScript,1734,programming,2020,1,1,9446000,18.356976,0
1,AE,JavaScript,2020-Q2,AE-JavaScript,2299,programming,2020,2,1,9446000,24.338344,0
2,AE,JavaScript,2020-Q3,AE-JavaScript,2366,programming,2020,3,1,9446000,25.047639,0
3,AE,JavaScript,2020-Q4,AE-JavaScript,2468,programming,2020,4,1,9446000,26.127461,0
4,AE,JavaScript,2021-Q1,AE-JavaScript,2544,programming,2021,5,1,9446000,26.932035,0
...,...,...,...,...,...,...,...,...,...,...,...,...
25595,LS,Kotlin,2022-Q4,LS-Kotlin,0,NaN,2022,12,1,2098000,0.0,1
25596,LS,Kotlin,2023-Q1,LS-Kotlin,0,NaN,2023,13,1,2098000,0.0,1
25597,LS,Kotlin,2023-Q2,LS-Kotlin,0,NaN,2023,14,1,2098000,0.0,1
25598,LS,Kotlin,2023-Q3,LS-Kotlin,0,NaN,2023,15,1,2098000,0.0,1


**Treatment**

In [20]:
balanced_df['Treatment'] = balanced_df.gpt_available * balanced_df.post_gpt
balanced_df

,iso2_code,language,year_quarter,unique_id,num_pushers,language_type,year,quarter,gpt_available,population,num_pushers_pc,post_gpt,Treatment
0,AE,JavaScript,2020-Q1,AE-JavaScript,1734,programming,2020,1,1,9446000,18.356976,0,0
1,AE,JavaScript,2020-Q2,AE-JavaScript,2299,programming,2020,2,1,9446000,24.338344,0,0
2,AE,JavaScript,2020-Q3,AE-JavaScript,2366,programming,2020,3,1,9446000,25.047639,0,0
3,AE,JavaScript,2020-Q4,AE-JavaScript,2468,programming,2020,4,1,9446000,26.127461,0,0
4,AE,JavaScript,2021-Q1,AE-JavaScript,2544,programming,2021,5,1,9446000,26.932035,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25595,LS,Kotlin,2022-Q4,LS-Kotlin,0,NaN,2022,12,1,2098000,0.0,1,1
25596,LS,Kotlin,2023-Q1,LS-Kotlin,0,NaN,2023,13,1,2098000,0.0,1,1
25597,LS,Kotlin,2023-Q2,LS-Kotlin,0,NaN,2023,14,1,2098000,0.0,1,1
25598,LS,Kotlin,2023-Q3,LS-Kotlin,0,NaN,2023,15,1,2098000,0.0,1,1


In [21]:
unique_values = balanced_df['unique_id'].nunique()
print(unique_values)

1600


### Check Variables

<Axes: >

In [22]:
balanced_df.gpt_available.hist()

In [23]:
balanced_df.to_csv("../output/data.csv")

In [24]:
# Suponiendo que df es tu DataFrame y ya ha sido cargado

# Agrupar por año y lenguaje y calcular la suma de num_pushers
yearly_language_pushers = data2.groupby(['year', 'language'])['num_pushers'].sum().reset_index()

# Calcular el porcentaje de pushers por año
yearly_language_pushers['percentage'] = yearly_language_pushers.groupby('year')['num_pushers'].transform(lambda x: (x / x.sum()) * 100)

# Ordenar los lenguajes por porcentaje total a lo largo de todos los años
total_percentage = yearly_language_pushers.groupby('language')['percentage'].sum().sort_values(ascending=False).index
yearly_language_pushers['language'] = pd.Categorical(yearly_language_pushers['language'], categories=total_percentage, ordered=True)
yearly_language_pushers = yearly_language_pushers.sort_values(['language', 'year'])

# Configuración del gráfico
fig, ax = plt.subplots(figsize=(20, 10))
colors = sns.color_palette('tab10', len(yearly_language_pushers['year'].unique()))

# Crear un gráfico de barras por cada año
years = yearly_language_pushers['year'].unique()
bar_width = 0.22
bar_positions = np.arange(len(total_percentage))

for i, year in enumerate(years):
    data = yearly_language_pushers[yearly_language_pushers['year'] == year]
    bars = ax.bar(bar_positions + i * bar_width, data['percentage'], bar_width, label=f'Año {year}', color=colors[i], edgecolor='black')

    # Añadir texto en cada barra
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width() / 2, yval + 0.1, f'{yval:.1f}%', ha='center', va='bottom', color='black', fontweight='bold', fontsize=8)

# Título y etiquetas
ax.set_title('', fontsize=22, color='black', fontweight='bold', pad=20)
ax.set_xlabel('Lenguaje de Programación', fontsize=16, color='black', fontweight='bold', labelpad=15)
ax.set_ylabel('Porcentaje (%)', fontsize=16, color='black', fontweight='bold', labelpad=15)

# Configuraciones de ejes y fondo
ax.set_facecolor('white')  # Fondo blanco para más claridad
fig.set_facecolor('white')
ax.tick_params(colors='black', which='both')  # Color de los ticks
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')
ax.set_xticks(bar_positions + bar_width * (len(years) - 1) / 2)
ax.set_xticklabels(total_percentage, rotation=0, ha='center', fontsize=12, fontweight='bold')

# Rejilla sutil
ax.grid(True, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)

# Añadir leyenda
ax.legend(title='Año', fontsize=14, title_fontsize=16, loc='upper right')

# Añadir la fuente y otros detalles a la derecha
fig.text(0.95, -0.01, 'Fuente: GitHub Innovation Graph', ha='right', va='bottom', fontsize=14, color='black')
fig.text(0.95, -0.05, 'Número de observaciones: 17675', ha='right', va='bottom', fontsize=14, color='black')
fig.text(0.95, -0.09, 'Cantidad de lenguajes de programación: 10', ha='right', va='bottom', fontsize=14, color='black')
fig.text(0.95, -0.13, 'Número de años: 4', ha='right', va='bottom', fontsize=14, color='black')

# Ajuste del diseño
plt.tight_layout(rect=[0, 0, 0.95, 1])  # Ajustar el diseño para dejar espacio para el texto
plt.show()

In [25]:
# Asegúrate de que 'data' es tu DataFrame y 'language' es la columna correcta
distribucion_porcentual = data2['language'].value_counts(normalize=True) * 100

# Configuración del gráfico
fig, ax = plt.subplots(figsize=(15, 8))
colors = plt.cm.tab20(np.arange(len(distribucion_porcentual))/len(distribucion_porcentual))  # Colores distintos para cada barra

# Crear barras
bars = ax.bar(distribucion_porcentual.index, distribucion_porcentual, color=colors, width=0.4, edgecolor='black')

# Añadir texto en cada barra
for bar in bars:
    yval = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2, yval + 0.5, f'{yval:.1f}%', ha='center', va='bottom', color='black', fontweight='bold', fontsize=10)

# Título y etiquetas
ax.set_title('Distribución de Lenguajes de Programación', fontsize=20, color='black', fontweight='bold')
ax.set_xlabel('Lenguaje de Programación', fontsize=14, color='black', fontweight='bold')
ax.set_ylabel('Porcentaje (%)', fontsize=14, color='black', fontweight='bold')

# Configuraciones de ejes y fondo
ax.set_facecolor('white')  # Fondo blanco para más claridad
fig.set_facecolor('white')
ax.tick_params(colors='black', which='both')  # Color de los ticks
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('black')
ax.spines['bottom'].set_color('black')

# Rejilla sutil
ax.grid(True, color='gray', linestyle='--', linewidth=0.5, alpha=0.5)

# Mostrar gráfico
plt.tight_layout()
plt.show()

In [26]:
# Aumentar el límite máximo de filas
alt.data_transformers.disable_max_rows()
alt.data_transformers.enable('json')

# Suponiendo que 'data' es tu DataFrame
data_aggregated = balanced_df.groupby('language').size().reset_index(name='counts')
data_aggregated['percentage'] = (data_aggregated['counts'] / data_aggregated['counts'].sum()) * 100


In [27]:
alt.Chart(data_aggregated).mark_bar()

c:\PYTHON\Python39\lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [28]:
# Convertir 'num_pushers' a numérico si es necesario
balanced_df['num_pushers_pc'] = pd.to_numeric(balanced_df['num_pushers_pc'], errors='coerce')

# Agregar datos por 'year_quarter' y 'language'
data_aggregated = balanced_df.groupby(['year_quarter', 'language'], as_index=False).agg(
    total_pushers=('num_pushers_pc', 'sum')
).rename(columns={'total_pushers': 'num_pushers_pc'})

In [29]:
data_aggregated

,year_quarter,language,num_pushers_pc
0,2020-Q1,C#,955.487976
1,2020-Q1,C++,995.988474
2,2020-Q1,Java,1778.005322
3,2020-Q1,JavaScript,4725.716388
4,2020-Q1,Kotlin,226.767866
...,...,...,...
155,2023-Q4,PHP,1248.937211
156,2023-Q4,Python,5593.741186
157,2023-Q4,Ruby,989.784309
158,2023-Q4,Swift,496.854339


In [30]:
data_aggregated.info() # Estos valores contabilizan a 10 lenguajes de programación en 16 trimestres

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year_quarter    160 non-null    object 
 1   language        160 non-null    object 
 2   num_pushers_pc  160 non-null    float64
dtypes: float64(1), object(2)
memory usage: 3.9+ KB


In [40]:
# Crear el gráfico principal
line_chart = alt.Chart(data_aggregated, height=450, width=600).mark_line(point=True, color='darkblue').encode(
    y=alt.Y('num_pushers_pc', title='Cantidad de pushes por cada 100 mil habitantes'),
    x=alt.X('year_quarter', title='Trimestre del Año'),
    color=alt.Color('language', title='Lenguaje de Programación'),
    tooltip=['num_pushers_pc', 'year_quarter', 'language']
).interactive().properties(
    title=''
)


# Crear el DataFrame con varias líneas de texto
text_data = pd.DataFrame({
    'text': [
        'Número de años: 4',
        'Número de observaciones: 160',
        'Cantidad de lenguajes de programación: 10',
        'Fuente: GIT Innovation Graph'
    ],
    'y': [20, 35, 50, 65]  # Ajusta estos valores para la posición vertical de cada línea
})

# Crear el texto de la fuente y las estadísticas
source_text = alt.Chart(text_data).mark_text(
    align='right',
    baseline='bottom',
    dx=-5,  # Ajusta este valor para mover el texto a la izquierda o derecha
    fontSize=12,
    color='black'
).encode(
    x=alt.value(640),  # posición x para alinear a la derecha (ajusta según sea necesario)
    y=alt.Y('y:Q', axis=None),
    text='text:N'
).properties(
    width=650,
    height=80  # Ajusta el alto para acomodar todas las líneas de texto
)

# Combinar el gráfico principal con el texto de la fuente y las estadísticas
final_chart = alt.vconcat(
    line_chart,
    source_text
).configure_concat(
    spacing=5
).configure_view(
    strokeWidth=0
)

final_chart.display()

c:\PYTHON\Python39\lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\PYTHON\Python39\lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\PYTHON\Python39\lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.VConcatChart(...)

In [32]:
data_con_gpt = balanced_df[balanced_df['gpt_available'] == 1]

In [33]:
# Aumentar el límite máximo de filas
alt.data_transformers.disable_max_rows()
alt.data_transformers.enable('json')

# Suponiendo que 'data' es tu DataFrame
data_agg_1 = data_con_gpt.groupby('language').size().reset_index(name='counts')
data_agg_1['percentage'] = (data_agg_1['counts'] / data_agg_1['counts'].sum()) * 100

alt.Chart(data_agg_1).mark_bar()


c:\PYTHON\Python39\lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [34]:
# Convertir 'num_pushers' a numérico si es necesario
data_con_gpt['num_pushers_pc'] = pd.to_numeric(data_con_gpt['num_pushers_pc'], errors='coerce')

# Agregar datos por 'year_quarter' y 'language'
data_agg_1 = data_con_gpt.groupby(['year_quarter', 'language'], as_index=False).agg(
    total_pushers=('num_pushers_pc', 'sum')
).rename(columns={'total_pushers': 'num_pushers_pc'})

C:\Users\ronco\AppData\Local\Temp\ipykernel_19924\3818098338.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_con_gpt['num_pushers_pc'] = pd.to_numeric(data_con_gpt['num_pushers_pc'], errors='coerce')


In [35]:

# Crear el gráfico principal
line_chart = alt.Chart(data_agg_1, height=450, width=600).mark_line(point=True, color='darkblue').encode(
    y=alt.Y('num_pushers_pc', title='Cantidad de pushes por cada 100 mil habitantes'),
    x=alt.X('year_quarter', title='Trimestre del Año'),
    color=alt.Color('language', title='Lenguaje de Programación'),
    tooltip=['num_pushers_pc', 'year_quarter', 'language']
).interactive().properties(
    title=''
)


# Crear el DataFrame con varias líneas de texto
text_data = pd.DataFrame({
    'text': [
        'Número de años: 4',
        'Número de observaciones: 160',
        'Cantidad de lenguajes de programación: 10',
        'Fuente: GIT Innovation Graph'
    ],
    'y': [20, 35, 50, 65]  # Ajusta estos valores para la posición vertical de cada línea
})

# Crear el texto de la fuente y las estadísticas
source_text = alt.Chart(text_data).mark_text(
    align='right',
    baseline='bottom',
    dx=-5,  # Ajusta este valor para mover el texto a la izquierda o derecha
    fontSize=12,
    color='black'
).encode(
    x=alt.value(640),  # posición x para alinear a la derecha (ajusta según sea necesario)
    y=alt.Y('y:Q', axis=None),
    text='text:N'
).properties(
    width=650,
    height=80  # Ajusta el alto para acomodar todas las líneas de texto
)

# Combinar el gráfico principal con el texto de la fuente y las estadísticas
final_chart = alt.vconcat(
    line_chart,
    source_text
).configure_concat(
    spacing=5
).configure_view(
    strokeWidth=0
)

final_chart.display()


c:\PYTHON\Python39\lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\PYTHON\Python39\lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\PYTHON\Python39\lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.VConcatChart(...)

In [36]:
data_sin_gpt = balanced_df[balanced_df['gpt_available'] == 0]

In [37]:
# Aumentar el límite máximo de filas
alt.data_transformers.disable_max_rows()
alt.data_transformers.enable('json')

# Suponiendo que 'data' es tu DataFrame
data_agg_2 = data_con_gpt.groupby('language').size().reset_index(name='counts')
data_agg_2['percentage'] = (data_agg_2['counts'] / data_agg_2['counts'].sum()) * 100

alt.Chart(data_agg_2).mark_bar()


c:\PYTHON\Python39\lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [38]:
# Convertir 'num_pushers' a numérico si es necesario
data_sin_gpt['num_pushers_pc'] = pd.to_numeric(data_sin_gpt['num_pushers_pc'], errors='coerce')

# Agregar datos por 'year_quarter' y 'language'
data_agg_2 = data_sin_gpt.groupby(['year_quarter', 'language'], as_index=False).agg(
    total_pushers=('num_pushers_pc', 'sum')
).rename(columns={'total_pushers': 'num_pushers_pc'})

C:\Users\ronco\AppData\Local\Temp\ipykernel_19924\949310509.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sin_gpt['num_pushers_pc'] = pd.to_numeric(data_sin_gpt['num_pushers_pc'], errors='coerce')


In [39]:
# Crear el gráfico principal
line_chart = alt.Chart(data_agg_2, height=450, width=600).mark_line(point=True, color='darkblue').encode(
    y=alt.Y('num_pushers_pc', title='Cantidad de pushes por cada 100 mil habitantes'),
    x=alt.X('year_quarter', title='Trimestre del Año'),
    color=alt.Color('language', title='Lenguaje de Programación'),
    tooltip=['num_pushers_pc', 'year_quarter', 'language']
).interactive().properties(
    title=''
)


# Crear el DataFrame con varias líneas de texto
text_data = pd.DataFrame({
    'text': [
        'Número de años: 4',
        'Número de observaciones: 160',
        'Cantidad de lenguajes de programación: 10',
        'Fuente: GIT Innovation Graph'
    ],
    'y': [20, 35, 50, 65]  # Ajusta estos valores para la posición vertical de cada línea
})

# Crear el texto de la fuente y las estadísticas
source_text = alt.Chart(text_data).mark_text(
    align='right',
    baseline='bottom',
    dx=-5,  # Ajusta este valor para mover el texto a la izquierda o derecha
    fontSize=12,
    color='black'
).encode(
    x=alt.value(640),  # posición x para alinear a la derecha (ajusta según sea necesario)
    y=alt.Y('y:Q', axis=None),
    text='text:N'
).properties(
    width=650,
    height=80  # Ajusta el alto para acomodar todas las líneas de texto
)

# Combinar el gráfico principal con el texto de la fuente y las estadísticas
final_chart = alt.vconcat(
    line_chart,
    source_text
).configure_concat(
    spacing=5
).configure_view(
    strokeWidth=0
)

final_chart.display()

c:\PYTHON\Python39\lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\PYTHON\Python39\lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
c:\PYTHON\Python39\lib\site-packages\altair\utils\core.py:384: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.VConcatChart(...)